<a href="https://colab.research.google.com/github/RemyaRS/Importing-and-Exporting-Datasets/blob/main/WebScraping_and_Exporting_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing libraries required**

In [1]:
import numpy as np
# numpy perform a number of mathematical operations on arrays such as trigonometric, statistical, and algebraic routines

import pandas as pd
# pandas is used to analyze data

import requests
# requests module allows you to send HTTP requests using Python

from bs4 import BeautifulSoup
# Beautiful Soup is a library that makes it easy to scrape information from web pages
# BeautifulSoup's current version bs4, is a web scraping library

from time import sleep
#The sleep() function suspends execution of the current thread for a given number of seconds

from random import randint
#The randint Python function is a built-in method that lets you generate random integers using the random module

In [2]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"} # To get data in english format

#**Declare Feature Names** - Empty lists to append data

In [3]:
movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []

url of 1st static page -- https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating

url of 2nd static page -- https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=101&ref_=adv_nxt

url of 3rd static page -- https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=201&ref_=adv_nxt

and so on.............

only thr part 101, 102, 103.... are dynamic in page 2 ,3, 4,......... respectively

**creating an array of values and passing it in the url for dynamic webpages : to get 1 to 1000 records from each page having 100 records**

In [4]:
pages = np.arange(1,1000,100)

In [5]:
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))# to avoid heavy request in every 2 to 8 seconds 
    for store in movie_data:
        name = store.h3.a.text# Since movie name is present in a tag of h3 tag as a text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "NaN"
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})#for indexing .. to get vote and grosses... that are present in same line
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else 'NaN'
        gross.append(grosses)

        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NaN'
        description.append(description_)

In [6]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross" : gross})

In [7]:
movie_list

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross
0,The Shawshank Redemption,(1994),142 min,9.3,81,"2,602,075",$28.34M
1,The Godfather,(1972),175 min,9.2,100,"1,797,537",$134.97M
2,The Dark Knight,(2008),152 min,9.0,84,"2,573,513",$534.86M
3,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94,"1,786,976",$377.85M
4,Schindler's List,(1993),195 min,9.0,94,"1,323,232",$96.90M
...,...,...,...,...,...,...,...
995,Sabrina,(1954),113 min,7.6,72,"64,465",NaN
996,From Here to Eternity,(1953),118 min,7.6,85,"46,793",$30.50M
997,Snow White and the Seven Dwarfs,(1937),83 min,7.6,95,"196,306",$184.93M
998,The 39 Steps,(1935),86 min,7.6,93,"56,568",NaN


In [8]:
# #saving the data in excel format
movie_list.to_excel("Top 1000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movie_list.to_csv("Top 1000 IMDb movies.csv")

In [9]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [10]:
movie_list.to_csv('Movie_List_from_IMDB.csv')
!cp Movie_List_from_IMDB.csv "drive/My Drive/"